In [1]:
import rpyc
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

import time
import utils
from tqdm import tqdm_notebook as tqdm

%load_ext autoreload
%autoreload 2

from environment.system import System
from agent.tabular_q_learning import Agent as T_Agent

Actions:
- 0 (rotation), 1 (other rotation), 2 (move outwards), 3 (move inwards)

In [12]:
import collections

class Environment():
    def __init__(self, field_classifier, reward_classifier, get_reward_function, get_state_function, state_queue_len=10):
        self.system = System(brick_ip='ev3dev.local', get_state_mode='dict')
        self.field_classifier = utils.load_pickle(field_classifier)
        self.reward_classifier = utils.load_pickle(reward_classifier)
        self.opposite_action = {0:3,1:2,2:1,3:0}
        self.opposite_action = {i: self.action_space-i-1 for i in range(self.action_space)}
        
        self.on_field = True
        self.border_count = 0
#         self.color_on = color_on
        
        self.state_queue = collections.deque(maxlen=state_queue_len)
        self.reward_queue = collections.deque(maxlen=state_queue_len)
        
        self.get_reward_function = get_reward_function
        self.get_state_function = get_state_function
        
        for _ in range(state_queue_len):
            self._new_state()
        
    def reset(self):
        # stop current action
        self.system.reset()
        # Go to initial state

        # return state
#         return self.prepro([self.state])
      
    def go_to_init_state(self):
        print('#'*30)
        print('Going to Init')
        print('#'*30)
        self.system.go_to_init_state()
#         time.sleep(3)
    
    @staticmethod
    def _color_from_one_state(s):
        return s[:3]
        
    def _environment_checks(self):
        # access color information from the last measurement 
        # according to self.new_state() ordering
        color = self._color_state_for_classifier
        
        if self.field_classifier.predict(color) == [0]:
            print('I am outside')
            self.border_count += 1

            if self.on_field:
                self.system.perform_actions([self.opposite_action[a] for a in self.current_action])
                print('BOUNCIN!!1')
                time.sleep(1)
            self.on_field = False
        else:
            self.on_field = True
    
        if self.border_count == 3:
            self.go_to_init_state()
            self.border_count = 0
            
    @property
    def _color_state_for_classifier(self):
        return np.array([self._color_from_one_state(s) for s in list(self.state_queue)[-2:]]).reshape(1,-1)
            
    def _new_reward_approx(self):
        def transform_proba_into_reward_approx(proba):
            return np.max([0., 5. * (proba - 0.3)])
            
        # Predict propba
        if not self.on_field:
            return -10
        
        colors = self._color_state_for_classifier
#         r = (np.argmax(self.reward_classifier.predict_proba(x), axis = 1) == 1).sum()
        # sum the probabilities of black class and compute a function of it
        black_proba = self.reward_classifier.predict_proba(colors)[:,1][0]
        
        self.reward_queue.append(transform_proba_into_reward_approx(black_proba))
        
    def _new_state(self):
        s = self.system.get_state()
        color = s['cs'][0]
        top_pos = s['top'][0]
        bot_pos = s['bot'][0]
        self.state_queue.append((*color, top_pos, bot_pos))
        
    def _cycle(self, is_free_cycle):
        if not is_free_cycle:
            print("Performing action", self.current_action)
            self.system.perform_actions(self.current_action)
        
        # gets new states and puts it in the queue
        self._new_state()
        
        # environment specific checks like is it still in the field
        self._environment_checks()
        
        # calculate the reward 
        self._new_reward_approx()
        
    def step(self, action, free_cycles=5):
        self.current_action = action
        self._cycle(False)  
        for _ in range(free_cycles):
            self._cycle(True)
        return self.state, self.reward, False, {}
    
    @property
    def reward(self):
        return self.get_reward_function(self.reward_queue)
        
    @property
    def state(self):
        return self.get_state_function(self.state_queue)
    
    @property
    def action_space(self):
        return len(self.system.get_action_space()[0])

def get_reward_function(reward_queue):
    # how many last reward_approx to take into consideration
    rewards = list(reward_queue)[-3:]
    
    # linear weights
    weights = np.ones(shape=(len(rewards),))
    weights = [weight * i for i, weight in enumerate(weights)]
    rewards_weighted = [weight * p for weight, p in zip(weights, rewards)]
    
    # custom function deciding what should be rewarded (because reward_approx is based on probability)
#     black_threshold = 0.3
#     r = np.max([0, (np.sum(rewards_weighted)-(black_threshold*len(rewards_weighted))) * 5])
    
    return np.mean(rewards_weighted)
    
def get_state_function(state_queue):
#     print(state_queue)
    state = list(state_queue)
    s = state[-5:]
    return np.array(s)

In [15]:
env = Environment('./mlp_on_off.pickle','./mlp_white_black.pickle', 
                  state_queue_len = 10, 
                  get_reward_function = get_reward_function,
                  get_state_function = get_state_function)

{'sensors': {'bot': <environment.system.EnvSensor object at 0x000001D98F698668>, 'top': <environment.system.EnvSensor object at 0x000001D98F698400>, 'cs': <environment.system.EnvSensor object at 0x000001D98F698438>}, 'actionables': {'bot': <environment.system.EnvActionable object at 0x000001D98F698630>, 'top': <environment.system.EnvActionable object at 0x000001D98F6986A0>}}


In [ ]:
env.go_to_init_state()

In [ ]:
env.step([2,2])

In [ ]:
env.action_space

In [21]:
env.reset()
for i in range(100):
    a = np.random.randint(env.action_space, size = 2)
    print(env.step(list(a)))

Performing action [0, 1]
##############################
Going to Init
##############################
(array([[248, 209, 221,  15,  12],
       [247, 209, 222,  15,  12],
       [247, 208, 222,  15,  12],
       [247, 209, 222,  15,  12],
       [247, 209, 221,  15,  12]]), 0.0, False, {})
Performing action [1, 5]
(array([[250, 213, 226,  23,   2],
       [265, 228, 247,  30,  -3],
       [266, 231, 248,  34,  -7],
       [266, 230, 248,  37, -10],
       [266, 229, 248,  40, -14]]), 0.0, False, {})
Performing action [2, 4]
(array([[259, 223, 239,  44, -19],
       [257, 222, 238,  44, -20],
       [254, 212, 234,  44, -20],
       [254, 210, 232,  44, -20],
       [252, 209, 232,  44, -20]]), 0.0, False, {})
Performing action [7, 7]
(array([[274, 239, 259,  57,  -5],
       [210, 182, 192,  66,   6],
       [ 32,  27,  30,  76,  17],
       [263, 230, 247,  86,  28],
       [269, 232, 249,  95,  39]]), 0.3303110402589184, False, {})
Performing action [4, 3]
(array([[256, 216, 232,  98,

Performing action [0, 2]
(array([[ 96,  89,  97,  61,  18],
       [264, 227, 244,  56,  10],
       [261, 225, 240,  49,   2],
       [261, 225, 239,  42,  -6],
       [262, 226, 244,  35, -12]]), 0.0, False, {})
Performing action [7, 7]
(array([[263, 227, 245,  41,  -4],
       [265, 229, 246,  52,   7],
       [269, 233, 251,  64,  19],
       [ 29,  24,  27,  74,  30],
       [214, 192, 218,  84,  40]]), 1.803851320793043, False, {})
Performing action [7, 4]
(array([[266, 227, 245, 100,  66],
       [268, 233, 245, 106,  75],
       [ 65,  71,  36, 110,  84],
       [155, 123, 215, 115,  91],
       [261, 220, 241, 120,  98]]), 0.8364596448288394, False, {})
Performing action [3, 2]
(array([[260, 221, 239, 117,  96],
       [202, 152, 216, 112,  92],
       [248, 219, 210, 100,  82],
       [256, 220, 228,  95,  76],
       [248, 208, 220,  90,  70]]), 0.0, False, {})
Performing action [4, 3]
(array([[234, 194, 204,  82,  69],
       [236, 197, 207,  82,  70],
       [236, 196, 207

       [ 42,  37,  38,  79,  42]]), 2.7863301198948673, False, {})
Performing action [6, 1]
(array([[254, 214, 229,  75,  62],
       [245, 208, 218,  70,  67],
       [146, 118, 113,  66,  71],
       [ 29,  22,  22,  63,  75],
       [ 31,  22,  31,  62,  79]]), 3.4484074577130652, False, {})
Performing action [0, 0]
(array([[209, 174, 193,  54,  69],
       [249, 210, 220,  47,  58],
       [231, 192, 198,  40,  46],
       [245, 208, 216,  33,  36],
       [244, 206, 216,  25,  27]]), 0.0, False, {})
Performing action [4, 1]
(array([[238, 200, 210,  10,  25],
       [230, 193, 202,   8,  29],
       [233, 195, 204,   8,  30],
       [235, 196, 206,   8,  30],
       [235, 197, 206,   8,  30]]), 0.0, False, {})
Performing action [2, 6]
(array([[252, 213, 225,  21,  20],
       [261, 224, 239,  30,  15],
       [261, 224, 237,  36,  12],
       [263, 225, 242,  42,  10],
       [267, 230, 246,  47,   9]]), 0.0, False, {})
Performing action [0, 6]
(array([[256, 220, 236,  63,  -2],
  

In [ ]:
env.system.perform_actions([1,1])

In [20]:
env.reset()

In [ ]:
num_episodes = 30

# Make an Agent

q_table = T_Agent(4, learn_rate = .8, gamma =.95)

#create lists to contain total rewards and steps per episode
env.reset()
rewards = []

stop_flag = False
for i in range(num_episodes):
    # Decay the exploration
    q_table.explore_decay = i
    
    s = env.go_to_init_state()
    rAll = 0
    
    d = False
    j = 0
    #The Q-Table learning algorithm
    try:
        while j < 99:
            j+=1

            #Choose an action by greedily (with noise) picking from Q table
            a = q_table.next_action(s)
            print('Action',a)
            #Get new state and reward from environment
            s1,r,d,_ = env.step(a)
            print('\r   ', r)

            #Update Q-Table with new knowledge
            q_table.update(r, s1)

            rAll += r
            s = s1
            if d == True:
                break
    except KeyboardInterrupt:
#         stop_flag = True
        env.reset()
        break

    rewards.append(rAll)
    print('#'*10, 'End Episode', '#'*10)
    
print("Average score over last part " +  str(sum(rewards[-500:])/500))

In [ ]:
start = time.time()
print(env.state)
print(time.time()- start)

In [ ]:
env.reset()

In [ ]:
q_table.val_table.shape